In [ ]:
import numpy as np
import aerokit.common.defaultgas as defg
import aerokit.aero.Isentropic as Is
import aerokit.aero.ShockWave  as sw
import aerokit.aero.MassFlow   as mf
import aerokit.aero.nozzle     as nz
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams['font.size'] = 14
#
def new_plot(xlab, ylab):
    plt.figure(figsize=(15,8))
    plt.xlabel(xlab); plt.ylabel(ylab)
    plt.grid(which='major', linestyle='-', alpha=0.8)
    plt.grid(which='minor', linestyle=':', alpha=0.5)

## Definition of flow section

In [ ]:
target_AoAc = 6.
length      = 8.
#
# set an A/Ac law from a Mach evolution, ensure exit Mach number is consistent with As/Ac
Noz_x    = np.linspace(0., length, 200, endpoint=True)
ma_max   = mf.Mach_Sigma(target_AoAc, Mach=2.)
ma       = 1. + (ma_max-1.)*np.sin(.5*(Noz_x-1.)*np.pi/(length-1.))
Noz_AoAc = mf.Sigma_Mach(ma)
#
new_plot('$x$', 'Mach, $r=\sqrt{A/A_c}$')
plt.plot(Noz_x, ma, Noz_x, np.sqrt(Noz_AoAc))
#plt.plot(Noz_x, np.sqrt(Noz_AoAc))
#
coord_x = np.append(Noz_x, [length, 1.05*length])  # define x coordinate with 2 additional points

# Computation of limits of flow regimes

In [ ]:
NPR0, NPRsw, NPR1, Msub, Msh, Msup = nz._NPR_Ms_list(target_AoAc)
print ("NPR limits are ", NPR0, NPRsw, NPR1, "\nwith respective Mach ", Msub, Msh, Msup)
mf.Sigma_Mach(Is.Mach_PtPs(np.array(nz._NPR_Ms_list(2.)[:3:2])))

# Isentropic subsonic flow

In [ ]:
Ma_col = .7
#
_M  = mf.MachSub_Sigma(Noz_AoAc*mf.Sigma_Mach(Ma_col))
_M  = np.append(_M, [_M[-1], _M[-1]])  # add constant points
_Pt = 0.*coord_x + 1.
_Ps = _Pt/Is.PtPs_Mach(_M)
#
new_plot('$x$','Mach')
plt.plot(coord_x, _M)

In [ ]:
new_plot('$x$','$P_s$, $P_t$')
plt.plot(coord_x, _Pt, coord_x, _Ps)
plt.ylim(0, 1.05)

# Isentropic subsonic and choked flow

In [ ]:
_M  = mf.Mach_Sigma(Noz_AoAc, Mach=.05)
_M  = np.append(_M, [_M[-1], _M[-1]])  # add constant points
_Pt = 0.*coord_x + 1.
_Ps = _Pt/Is.PtPs_Mach(_M)
#
new_plot('$x$','Mach')
plt.plot(coord_x, _M)

In [ ]:
new_plot('$x$','$P_s$, $P_t$')
plt.plot(coord_x, _Pt, coord_x, _Ps)
plt.ylim(0, 1.05)

# Isentropic supersonic flow

In [ ]:
_Minit = .05 + (Noz_x > 1.)*2.   # set initial guess to .05 in convergent, 2. in diffuser
_M  = mf.Mach_Sigma(Noz_AoAc, Mach=_Minit)
_M  = np.append(_M, [_M[-1], _M[-1]])  # add constant points
_Pt = 0.*coord_x + 1.
_Ps = _Pt/Is.PtPs_Mach(_M)
# 
new_plot('$x$','Mach')
plt.plot(coord_x, _M)

In [ ]:
new_plot('$x$','$P_s$, $P_t$')
plt.plot(coord_x, _Pt, coord_x, _Ps)
plt.ylim(0, 1.05)

# Supersonic nozzle with normal shock at exit

In [ ]:
# this needs computation in previous cell 
Ptloss = sw.Pi_ratio(Msup)
_M  = mf.Mach_Sigma(Noz_AoAc, Mach=_Minit)   # recompute _M to get right array size
_M  = np.append(_M, [Msh, Msh])              # add constant points, Msh computed in previous cells
_Pt = np.append(0.*Noz_x + 1., [Ptloss, Ptloss]);
_Ps = _Pt/Is.PtPs_Mach(_M)
#
new_plot('$x$','Mach')
plt.plot(coord_x, _M)

In [ ]:
new_plot('$x$','$P_s$, $P_t$')
plt.plot(coord_x, _Pt, coord_x, _Ps)
plt.ylim(0, 1.05)

# Supersonic nozzle with normal shock in diffuser

In [ ]:
pos = .5                            # 0 to 1 parameter to control shock position
NPR = NPR0*(1.-pos)+NPRsw*pos
#
# this needs computation of _Minit in previous cell 
_M  = mf.Mach_Sigma(Noz_AoAc, Mach=_Minit)
#
# analytical solution for Ms, losses and upstream Mach number of shock wave
Ms     = nz.Ms_from_AsAc_NPR(target_AoAc, NPR)
print (nz.Ms_from_AsAc_NPR(2.636, 1.5))
Ptloss = Is.PtPs_Mach(Ms)/NPR
Msh    = sw.Mn_Pi_ratio(Ptloss)
#
# redefine curves starting from 'ish' index (closest value of Msh in supersonic flow)
ish    = np.abs(_M-Msh).argmin()
_M[ish:] = mf.MachSub_Sigma(Noz_AoAc[ish:]*mf.Sigma_Mach(Ms)/target_AoAc)
_M  = np.append(_M, [Ms, Ms])  # add constant points
_Pt = np.append(0.*Noz_x + 1., [Ptloss, Ptloss]);
_Pt[ish:] = Ptloss
_Ps = _Pt/Is.PtPs_Mach(_M)
#
new_plot('$x$','Mach')
plt.plot(coord_x, _M)

In [ ]:
new_plot('$x$','$P_s$, $P_t$')
plt.plot(coord_x, _Pt, coord_x, _Ps)
plt.ylim(0, 1.05)

# Mach numbers in exit section and jet flow

In [ ]:
new_plot('$NPR$','$M_s$, $M_a$')
_npr=np.exp(np.linspace(0.01, np.log(NPR1*1.2), 200))
_ms =_npr.copy()
_ma =_npr.copy()
for i in np.arange(_npr.size):
    _ms[i] = nz.Ms_from_AsAc_NPR(target_AoAc, _npr[i])
    _ma[i] = nz.Madapt_from_AsAc_NPR(target_AoAc, _npr[i])
plt.semilogx(_npr,_ms, _npr, _ma)

In [ ]:
from IPython.core.display import HTML ; HTML(open("./custom.css", "r").read()) # notebook style